In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# WORKAROUND: Redefinir on_upload_change para soportar ipywidgets moderno
# ═══════════════════════════════════════════════════════════════════════════

def on_upload_change(change):
    """Callback cuando se sube un archivo - version corregida"""
    global uploaded_file_path
    
    with output_area:
        clear_output()
        
        if not upload_widget.value:
            print("No se selecciono archivo")
            return
        
        # Obtener archivo - soportar tuple y dict
        upload_value = upload_widget.value
        if isinstance(upload_value, tuple):
            uploaded_file = upload_value[0]
        else:
            uploaded_file = list(upload_value.values())[0]
        
        filename = uploaded_file["metadata"]["name"]
        content = uploaded_file["content"]
        
        print(f"Archivo seleccionado: {filename}")
        print(f"   Tamano: {len(content) / (1024*1024):.2f} MB")
        
        if not filename.lower().endswith(".pdf"):
            print("Error: Solo archivos PDF")
            return
        
        file_path = INPUT_DIR / filename
        
        if file_path.exists():
            print(f"'{filename}' ya existe, sobrescribiendo...")
        
        with open(file_path, "wb") as f:
            f.write(content)
        
        uploaded_file_path = file_path
        
        print(f"Guardado en: {file_path}")
        print()
        listar_pdfs_input()

# Re-registrar el callback
upload_widget.unobserve_all()
upload_widget.observe(on_upload_change, names="value")

print("WORKAROUND APLICADO: Ejecuta esta celda para habilitar carga de PDFs")


# Pipeline de Extracción de Especificaciones Técnicas desde PDF

**Objetivo:** Extraer rubros, materiales y equipos desde PDFs de especificaciones técnicas y exportar a Excel.

**Autor:** Senior Data/ML Engineer  
**Fecha:** 2026-01-28

---

## 📋 Contenido

1. [Setup y Configuración](#setup)
2. [Gestión de Archivos](#gestion-archivos) ⭐ NUEVO
3. [Ejemplo Básico: Pipeline End-to-End](#ejemplo-basico)
4. [Análisis de Resultados](#analisis)
5. [Ejemplo Avanzado: Múltiples PDFs](#batch)
6. [Debugging y Warnings](#debugging)
7. [Personalización](#personalizacion)

---

## 1. Setup y Configuración

### Importar módulos

In [ ]:
# Agregar src/ al path para importar módulos locales
import sys
from pathlib import Path

# Asegurarse de que estamos en la raíz del proyecto
if Path.cwd().name == 'notebooks':
    import os
    os.chdir('..')

print(f"Directorio actual: {Path.cwd()}")

In [ ]:
# Imports principales
from src.pipeline import run_pipeline, process_multiple_pdfs
from src.utils.logger import configure_logging, get_logger
from src.models.schemas import PipelineResult

import pandas as pd
from rich import print as rprint
from rich.table import Table
from rich.console import Console

console = Console()

# Configurar logging
configure_logging(level="INFO", json_logs=False)
logger = get_logger(__name__)

print("✅ Módulos importados correctamente")

### Verificar instalaciones críticas

In [ ]:
# Verificar Tesseract
from src.ocr.tesseract_ocr import test_tesseract_installation, get_available_languages

if test_tesseract_installation():
    print("✅ Tesseract instalado correctamente")
    langs = get_available_languages()
    print(f"Idiomas disponibles: {langs}")
    
    if 'spa' in langs:
        print("✅ Idioma español (spa) disponible")
    else:
        print("⚠️ ADVERTENCIA: Idioma español no instalado. OCR puede fallar.")
else:
    print("❌ Tesseract NO instalado.")
    print("   Para instalar: bash install_tesseract.sh")
    print("   O ver: INSTALL_TESSERACT_WSL.md")

---

## 2. Gestión de Archivos ⭐ NUEVO

### 2.1 Preparar Directorios

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# GESTIÓN DE ARCHIVOS - Funciones de utilidad
# ═══════════════════════════════════════════════════════════════════════════

import shutil
from datetime import datetime

# Directorios
INPUT_DIR = Path("data/input")
OUTPUT_DIR = Path("data/output")

# Crear si no existen
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def listar_pdfs_input():
    """Lista PDFs en data/input/"""
    pdfs = sorted(INPUT_DIR.glob("*.pdf"), key=lambda p: p.stat().st_mtime, reverse=True)
    if pdfs:
        print(f"📂 PDFs en {INPUT_DIR}:")
        for idx, pdf in enumerate(pdfs, 1):
            size_mb = pdf.stat().st_size / (1024 * 1024)
            mtime = datetime.fromtimestamp(pdf.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
            print(f"  {idx}. {pdf.name:40} ({size_mb:6.2f} MB)  [{mtime}]")
        return pdfs
    else:
        print(f"📂 {INPUT_DIR} está vacío (no hay PDFs)")
        return []

def limpiar_input_dir(confirmar=True):
    """Limpia todos los PDFs de data/input/"""
    pdfs = list(INPUT_DIR.glob("*.pdf"))
    
    if not pdfs:
        print("✓ Directorio ya está vacío")
        return
    
    if confirmar:
        print(f"⚠️  Se eliminarán {len(pdfs)} archivo(s):")
        for pdf in pdfs:
            print(f"   - {pdf.name}")
        
        respuesta = input("\n¿Continuar? (s/N): ").strip().lower()
        if respuesta not in ['s', 'si', 'sí', 'yes', 'y']:
            print("❌ Cancelado")
            return
    
    for pdf in pdfs:
        pdf.unlink()
        print(f"   🗑️  Eliminado: {pdf.name}")
    
    print(f"✅ Directorio limpiado")

# Mostrar estado inicial
print("📊 ESTADO ACTUAL DE DIRECTORIOS")
print("=" * 70)
listar_pdfs_input()
print()
print("💡 Funciones disponibles:")
print("   • listar_pdfs_input() - Ver PDFs disponibles")
print("   • limpiar_input_dir() - Limpiar directorio input")

### 2.2 Upload de PDF (con Widget Interactivo)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# UPLOAD DE PDF - Widget Interactivo (VERSIÓN ROBUSTA)
# ═══════════════════════════════════════════════════════════════════════════

from ipywidgets import FileUpload, Button, Output, VBox, HBox, HTML
from IPython.display import display, clear_output

# Widget de upload
upload_widget = FileUpload(
    accept='.pdf',
    multiple=False,
    description='📤 Seleccionar PDF:',
    button_style='primary',
    style={'description_width': 'initial'}
)

# Output para mensajes
output_area = Output()

# Variable global para almacenar el path del archivo
uploaded_file_path = None

def extract_file_data(upload_value):
    '''
    Extrae filename y content de forma robusta desde upload_value.
    Compatible con ipywidgets >= 8.0 (moderno) y <= 7.x (legacy).
    '''
    # Paso 1: Obtener el uploaded_file dict
    uploaded_file = None

    if isinstance(upload_value, tuple):
        # Formato nuevo: tuple con un elemento
        if len(upload_value) > 0:
            uploaded_file = upload_value[0]
    elif isinstance(upload_value, dict):
        # Formato legacy: dict con keys como filenames
        if upload_value:
            uploaded_file = list(upload_value.values())[0]

    if not uploaded_file or not isinstance(uploaded_file, dict):
        print("❌ Formato de upload inesperado")
        return None, None

    # Paso 2: Extraer filename (múltiples estrategias con fallback)
    filename = None

    # Estrategia 1: Formato moderno (directo)
    if "name" in uploaded_file:
        filename = uploaded_file["name"]

    # Estrategia 2: Formato legacy (metadata.name)
    elif "metadata" in uploaded_file and isinstance(uploaded_file["metadata"], dict):
        filename = uploaded_file["metadata"].get("name")

    # Estrategia 3: Buscar en todas las claves (defensivo)
    else:
        print(f"⚠️  Formato no reconocido. Keys disponibles: {list(uploaded_file.keys())}")

        for key, value in uploaded_file.items():
            if isinstance(value, dict) and "name" in value:
                filename = value["name"]
                print(f"✓ Filename encontrado en '{key}.name'")
                break

    # Paso 3: Extraer content
    content = uploaded_file.get("content")

    # Validación final
    if not filename:
        print("❌ No se pudo extraer el nombre del archivo")
        print(f"   Keys disponibles: {list(uploaded_file.keys())}")
        return None, None

    if not content:
        print("❌ No se pudo extraer el contenido del archivo")
        return None, None

    return filename, content

def on_upload_change(change):
    '''Callback cuando se sube un archivo - VERSIÓN ROBUSTA'''
    global uploaded_file_path

    with output_area:
        clear_output()

        if not upload_widget.value:
            print("❌ No se seleccionó archivo")
            return

        # Extraer filename y content de forma robusta
        filename, content = extract_file_data(upload_widget.value)

        if not filename or not content:
            return

        print(f"📄 Archivo seleccionado: {filename}")
        print(f"   Tamaño: {len(content) / (1024*1024):.2f} MB")

        # Validar PDF
        if not filename.lower().endswith('.pdf'):
            print("❌ Error: Solo archivos PDF")
            return

        # Guardar
        file_path = INPUT_DIR / filename

        if file_path.exists():
            print(f"⚠️  '{filename}' ya existe, sobrescribiendo...")

        try:
            with open(file_path, 'wb') as f:
                f.write(content)

            uploaded_file_path = file_path

            print(f"✅ Guardado en: {file_path}")
            print()
            listar_pdfs_input()

        except Exception as e:
            print(f"❌ Error al guardar: {e}")

upload_widget.observe(on_upload_change, names='value')

# Botón de limpiar
btn_limpiar = Button(
    description='🗑️ Limpiar Input',
    button_style='warning',
    tooltip='Eliminar todos los PDFs de data/input/'
)

def on_limpiar_click(b):
    with output_area:
        clear_output()
        limpiar_input_dir(confirmar=False)
        print()
        listar_pdfs_input()

btn_limpiar.on_click(on_limpiar_click)

# UI
print("📤 UPLOAD DE PDF")
print("=" * 70)
display(HTML("<div style='background: #f0f8ff; padding: 10px; border-left: 4px solid #2196F3;'>"
             "<b>📌 Instrucciones:</b><br>"
             "1️⃣ Click en <b>'Seleccionar PDF'</b> y elige tu archivo<br>"
             "2️⃣ El archivo se guarda automáticamente en <code>data/input/</code><br>"
             "3️⃣ Usa <b>'🗑️ Limpiar Input'</b> si quieres empezar de cero"
             "</div>"))
print()
display(VBox([
    HBox([upload_widget, btn_limpiar]),
    output_area
]))

### 2.3 Seleccionar PDF a Procesar

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# SELECCIÓN DE PDF
# ═══════════════════════════════════════════════════════════════════════════

def seleccionar_pdf_input():
    """
    Selecciona un PDF de data/input/ de forma interactiva.
    Si hay solo 1, lo selecciona automáticamente.
    Si hay múltiples, pide al usuario que elija.
    """
    pdfs = listar_pdfs_input()
    
    if not pdfs:
        print()
        print("❌ No hay PDFs disponibles")
        print("   → Ejecuta la celda anterior para subir un PDF")
        return None
    
    if len(pdfs) == 1:
        print()
        print(f"✅ Único PDF disponible: {pdfs[0].name}")
        return pdfs[0]
    
    # Múltiples PDFs
    print()
    print("Selecciona un PDF (ingresa el número):")
    
    while True:
        try:
            seleccion = input("Número [1]: ").strip()
            
            if not seleccion:
                seleccion = "1"
            
            idx = int(seleccion) - 1
            
            if 0 <= idx < len(pdfs):
                pdf_seleccionado = pdfs[idx]
                print(f"✅ Seleccionado: {pdf_seleccionado.name}")
                return pdf_seleccionado
            else:
                print(f"❌ Número inválido. Entre 1 y {len(pdfs)}")
        
        except ValueError:
            print("❌ Ingresa un número válido")
        except KeyboardInterrupt:
            print("\n❌ Cancelado")
            return None

# Seleccionar PDF
print("📄 SELECCIÓN DE PDF")
print("=" * 70)
PDF_INPUT = seleccionar_pdf_input()

if PDF_INPUT:
    print()
    print("✅ PDF SELECCIONADO PARA PROCESAR:")
    print("=" * 70)
    print(f"   Archivo: {PDF_INPUT.name}")
    print(f"   Ruta:    {PDF_INPUT}")
    print(f"   Tamaño:  {PDF_INPUT.stat().st_size / (1024*1024):.2f} MB")
    print()
    print("✅ Listo para pipeline")
else:
    print()
    print("⚠️  No se seleccionó PDF")
    print("   Ejecuta la celda anterior para cargar uno")

### 2.4 Configurar Nombre de Salida

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CONFIGURAR OUTPUT
# ═══════════════════════════════════════════════════════════════════════════

if PDF_INPUT:
    # Generar nombre automático
    OUTPUT_NOMBRE = PDF_INPUT.stem + "_resultado.xlsx"
    OUTPUT_PATH = OUTPUT_DIR / OUTPUT_NOMBRE
    
    print("📊 CONFIGURACIÓN DE SALIDA")
    print("=" * 70)
    print(f"   Input:  {PDF_INPUT.name}")
    print(f"   Output: {OUTPUT_NOMBRE}")
    print(f"   Ruta:   {OUTPUT_PATH}")
    print()
    
    if OUTPUT_PATH.exists():
        print(f"⚠️  '{OUTPUT_NOMBRE}' ya existe (será sobrescrito)")
    else:
        print("✅ Archivo nuevo")
else:
    print("❌ Primero selecciona un PDF en la celda anterior")

### 2.5 Resumen Pre-Ejecución

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# RESUMEN PRE-EJECUCIÓN
# ═══════════════════════════════════════════════════════════════════════════

from IPython.display import HTML, display

print()
print("🚀 RESUMEN DE CONFIGURACIÓN")
print("=" * 70)
print()

if PDF_INPUT and OUTPUT_PATH:
    display(HTML(
        "<div style='background: #e8f5e9; padding: 15px; border-left: 4px solid #4CAF50;'>"
        f"<b>📥 INPUT:</b><br>"
        f"&nbsp;&nbsp;• Archivo: <code>{PDF_INPUT.name}</code><br>"
        f"&nbsp;&nbsp;• Tamaño: {PDF_INPUT.stat().st_size / (1024*1024):.2f} MB<br>"
        f"&nbsp;&nbsp;• Ruta: <code>{PDF_INPUT}</code><br><br>"
        f"<b>📤 OUTPUT:</b><br>"
        f"&nbsp;&nbsp;• Archivo: <code>{OUTPUT_PATH.name}</code><br>"
        f"&nbsp;&nbsp;• Ruta: <code>{OUTPUT_PATH}</code><br>"
        f"&nbsp;&nbsp;• Estado: {'Sobrescribirá existente' if OUTPUT_PATH.exists() else 'Nuevo'}<br><br>"
        f"<b>⏰ Timestamp:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}<br><br>"
        "<b style='color: #4CAF50;'>✅ LISTO PARA EJECUTAR PIPELINE</b>"
        "</div>"
    ))
    
    print()
    print("⬇️  Siguiente celda: Ejecutar run_pipeline()")

else:
    display(HTML(
        "<div style='background: #ffebee; padding: 15px; border-left: 4px solid #f44336;'>"
        "<b>❌ CONFIGURACIÓN INCOMPLETA</b><br><br>"
        f"{'❌ Falta seleccionar PDF de entrada<br>' if not PDF_INPUT else ''}"
        f"{'❌ Falta configurar ruta de salida<br>' if not OUTPUT_PATH else ''}"
        "<br>Ejecuta las celdas anteriores primero"
        "</div>"
    ))

---

## 3. Ejemplo Básico: Pipeline End-to-End

### Ejecutar Pipeline

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# EJECUTAR PIPELINE
# ═══════════════════════════════════════════════════════════════════════════

# Verificar que tenemos PDF seleccionado
if not PDF_INPUT or not OUTPUT_PATH:
    print("❌ ERROR: Primero debes seleccionar un PDF (ejecuta celdas anteriores)")
else:
    print("🚀 Iniciando pipeline...")
    print("=" * 70)
    print()
    
    # Ejecutar pipeline
    result = run_pipeline(
        pdf_path=PDF_INPUT,
        output_path=OUTPUT_PATH,
        force_ocr=False,  # Auto-detectar si requiere OCR
        ocr_lang='spa',   # Idioma español
        ocr_dpi=300,      # DPI para OCR
        apply_excel_formatting=True
    )
    
    print()
    print("=" * 70)
    print(f"✅ Pipeline completado exitosamente")
    print(f"📊 Excel generado: {OUTPUT_PATH}")
    print()
    print("⬇️  Siguiente: Analizar resultados en las celdas siguientes")

---

## 4. Análisis de Resultados

### Resumen de metadatos

In [ ]:
# Mostrar metadatos del documento
if 'result' in globals():
    metadata = result.metadata
    
    table = Table(title="📊 Resumen del Procesamiento")
    table.add_column("Campo", style="cyan")
    table.add_column("Valor", style="green")
    
    table.add_row("Archivo", metadata.filename)
    table.add_row("Total Páginas", str(metadata.total_pages))
    table.add_row("Tipo de Documento", metadata.tipo_documento)
    table.add_row("Páginas con OCR", str(len(metadata.pages_with_ocr)))
    table.add_row("Total Rubros", str(metadata.total_rubros))
    table.add_row("Total Recursos", str(metadata.total_recursos))
    table.add_row("Total Warnings", str(metadata.total_warnings))
    table.add_row("Success Rate", f"{result.success_rate * 100:.1f}%")
    
    console.print(table)
else:
    print("❌ Ejecuta primero la celda de pipeline")

### Explorar rubros extraídos

In [ ]:
# Convertir rubros a DataFrame
if 'result' in globals() and result.rubros:
    rubros_data = [
        {
            'codigo': r.codigo,
            'descripcion': r.descripcion[:50] + '...' if len(r.descripcion) > 50 else r.descripcion,
            'unidad': r.unidad,
            'confidence': f"{r.confidence:.2f}"
        }
        for r in result.rubros[:10]
    ]
    
    df_rubros = pd.DataFrame(rubros_data)
    print(f"📋 Primeros {len(df_rubros)} rubros extraídos:")
    display(df_rubros)
else:
    print("❌ No hay rubros para mostrar")

### Explorar recursos extraídos

In [ ]:
# Convertir recursos a DataFrame
if 'result' in globals() and result.recursos:
    recursos_data = [
        {
            'rubro_id': r.rubro_id,
            'tipo': r.tipo,
            'nombre': r.nombre[:40] + '...' if len(r.nombre) > 40 else r.nombre,
            'unidad': r.unidad or 'N/A'
        }
        for r in result.recursos[:15]
    ]
    
    df_recursos = pd.DataFrame(recursos_data)
    print(f"🔧 Primeros {len(df_recursos)} recursos extraídos:")
    display(df_recursos)
else:
    print("❌ No hay recursos para mostrar")

### Distribución de tipos de recursos

In [ ]:
# Contar recursos por tipo
if 'result' in globals() and result.recursos:
    from collections import Counter
    
    tipos_count = Counter([r.tipo for r in result.recursos])
    
    table = Table(title="🔢 Distribución de Recursos por Tipo")
    table.add_column("Tipo", style="cyan")
    table.add_column("Cantidad", style="magenta")
    table.add_column("Porcentaje", style="green")
    
    total = sum(tipos_count.values())
    for tipo, count in tipos_count.most_common():
        pct = (count / total) * 100 if total > 0 else 0
        table.add_row(tipo, str(count), f"{pct:.1f}%")
    
    console.print(table)
else:
    print("❌ No hay recursos para analizar")

---

## 5. Debugging y Warnings

### Analizar warnings por severidad

In [ ]:
# Warnings por severidad
if 'result' in globals():
    warnings_high = result.get_warnings_by_severity('HIGH')
    warnings_medium = result.get_warnings_by_severity('MEDIUM')
    warnings_low = result.get_warnings_by_severity('LOW')
    
    print(f"🔴 HIGH: {len(warnings_high)} warnings")
    print(f"🟡 MEDIUM: {len(warnings_medium)} warnings")
    print(f"⚪ LOW: {len(warnings_low)} warnings")
    
    if warnings_high:
        print("\n🔴 Warnings HIGH (críticos):")
        for w in warnings_high[:5]:
            print(f"  • Página {w.page}: {w.message}")
            if w.snippet:
                print(f"    Snippet: {w.snippet[:80]}...")
else:
    print("❌ Ejecuta primero el pipeline")

---

## 6. Ejemplo Avanzado: Múltiples PDFs (Batch)

Procesar todos los PDFs en `data/input/` en batch.

In [ ]:
# Batch processing
results_batch = process_multiple_pdfs(
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    pattern="*.pdf",
    force_ocr=False,
    ocr_lang='spa'
)

# Resumen
print(f"\n✅ Batch completado: {len(results_batch)} archivos")

for filename, res in results_batch.items():
    if res:
        print(f"  • {filename}: {res.metadata.total_rubros} rubros, {res.metadata.total_recursos} recursos")
    else:
        print(f"  • {filename}: ❌ ERROR")

---

## 🎯 Próximos Pasos

1. **Iterar sobre reglas de parseo:** Ajusta heurísticas en `src/parse/rubro_parser.py`
2. **Agregar keywords:** Actualiza `MATERIAL_INDICATORS` y `EQUIPO_INDICATORS`
3. **Mejorar OCR:** Experimenta con diferentes valores de DPI
4. **Exportar a otros formatos:** CSV, JSON, etc.
5. **Integrar con DB:** PostgreSQL para análisis

---

**¿Preguntas?** Ver [SPEC.md](../SPEC.md) para documentación completa.